<a href="https://colab.research.google.com/github/KanthiKiranS/KK-Repo/blob/master/Pheme_Veracity_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import fastai

In [ ]:
from fastai.text import * 

In [ ]:
def read_veracity(path_to_file, shuffle=False):
    def enc(col):
        if col['label'] == 2.0:
            return 'Unverified'
        elif col['label'] == 1.0:
            return 'True'
        elif col['label'] == 0.0:
            return 'False'

    df = pd.read_csv(path_to_file)
    df = df.drop(['Unnamed: 0', '0'], 1)
    df.columns = ['text', 'label']
    df['label'] = df.apply(enc, 1)
    if shuffle:
        df = df.sample(frac=1).reset_index(drop=True)

    return df.dropna().iloc[:, ::-1]

In [ ]:
# For veracity
train = read_veracity('modifiedTrain_New.csv', True)
test = read_veracity('modifiedTest_New.csv')

In [ ]:
train.to_csv('train.csv')
test.to_csv('test.csv')

In [ ]:
pd.concat([train['text'], test['text']]).reset_index(drop=True).to_csv('text.csv')

In [ ]:
data_lm = TextLMDataBunch.from_csv('', 'text.csv')
data_clas = TextClasDataBunch.from_csv('', 'train.csv', vocab=data_lm.train_ds.vocab, text_cols='text', label_cols='label', bs=32)

In [ ]:
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

In [ ]:
data_lm = load_data('', 'data_lm_export.pkl')
data_clas = load_data('', 'data_clas_export.pkl', bs=16)

In [ ]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.579438,4.286726,0.289058,03:36


In [ ]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.059246,3.978714,0.327834,04:13


In [ ]:
learn.predict("This is a review about", n_words=10)

'This is a review about barbaric reprisals against Jews its village http : /'

In [ ]:
learn.save_encoder('ft_enc')

In [ ]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (76440 items)
x: TextList
xxbos xxmaj its sad . “ @soulrevision : xxmaj the cop who shot # mikebrown is on paid leave . 
  xxmaj shoot a black kid , get a paid vacation . # wearetargets # xxmaj ferguson ”,xxbos @tracymo53321503 @mashable are you really justifying the murder of an unarmed kid ? o,xxbos @commentisfree @simonjenkins4 xxmaj mal wieder ein xxunk xxmaj xxunk ... ts ts .. https : / / t.co / xxunk # charliehebdo,xxbos @jfbegin xxmaj le tweet qui a xxunk la xxunk : https : / / t.co / xxunk 
  @nycjim,xxbos @tarekfatah @barackobama @david_cameron @justintrudeau xxmaj islam is the religion of xxup pi xxrep 20 s ! ! !
y: CategoryList
False,True,False,True,False
Path: .;

Valid: LabelList (19111 items)
x: TextList
xxbos @aclu_mo xxmaj great .,xxbos @morbidentree @nbcnews false flags can happen anywhere,xxbos xxup this-- > xxup rt : @songbird_uk : xxmaj out of all the cartoons we 've seen , i think this is the most poignant , the 

In [ ]:
data_clas.show_batch()

text,target
xxbos “ @wesleylowery : a few feet from where # michaelbrown was killed : # xxmaj ferguson http : / / t.co / xxunk ” \n “ @wesleylowery : a few feet from where # michaelbrown was killed : # xxmaj ferguson http : / / t.co / xxunk \n “ @wesleylowery : a few feet from where # michaelbrown was killed : # xxmaj ferguson http :,False
"xxbos xxup rip - 💜 💙 “ @pzfeed : xxup breaking xxup news : xxup one xxup soldier xxup has xxup died xxup after xxup ottawa xxup shooting , xxup suspect(s ) xxup still xxup at xxup large - http : / / t.co / xxunk ”",True
"xxbos xxmaj sad “ @buzzfeednews : xxmaj hostages xxmaj held xxmaj in xxmaj sydney xxmaj chocolate xxmaj shop xxmaj forced xxmaj to xxmaj hold xxmaj up xxmaj islamic xxmaj flag http : / / t.co / xxunk https : / / t.co / xxunk """,True
xxbos .@sarahandrews77 xxmaj there is xxup no increased xxup threat to # xxup uk = # xxup mi5 has xxup no xxup intelligence # xxup mi5 & & # xxup cobra merely xxmaj covering their xxmaj arse xxup in xxup case of attack !,False
xxbos xxmaj xxunk ! “ @buzzfeednews : 5 xxmaj hostages xxmaj escape xxmaj sydney xxmaj cafe xxmaj siege — xxmaj live xxmaj updates xxmaj from @buzzfeedoz http : / / t.co / xxunk t http : / / t.co / xxunk ”,True


In [ ]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.542910,0.494298,0.764429,03:37


In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3), moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.481693,0.427418,0.801266,04:28


In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, slice(2e-3/100, 2e-3), moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.429816,0.402454,0.816493,08:34
1,0.411481,0.357486,0.836953,09:36
2,0.357710,0.310161,0.860133,09:20
3,0.322792,0.278082,0.877348,09:17
4,0.288670,0.263059,0.883313,08:30
5,0.230527,0.254066,0.887761,09:24
6,0.228294,0.250667,0.888807,09:18
7,0.192035,0.251469,0.891162,09:29
8,0.190453,0.251479,0.892680,08:35
9,0.150501,0.251531,0.891319,08:42


In [ ]:
l = []
for text in test['text']:
    l.append(str(learn.predict(text)[0]))

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
accuracy_score(test['label'], l)

0.5344174649142348

In [ ]:
f1_score(test['label'], l, average='macro')

0.5309312090502681

In [ ]:
learn.export('veracity.pkl')